## Classifiers

分类器是从资产和时刻到分类输出（如字符串或整数标签）的函数：

![分类器函数](./classify_fun.png)

生成字符串输出的分类器的一个示例是证券的交换所ID。要创建此分类器，我们必须导入`Fundamentals.exchange_id`并使用最新属性来实例化我们的分类器：

In [1]:
from zipline.pipeline.factors import AverageDollarVolume, SimpleMovingAverage
from zipline.pipeline import Pipeline
from zipline.pipeline.data import USEquityPricing
from zipline.research import run_pipeline

In [2]:
from zipline.pipeline.fundamentals import Fundamentals

# Since the underlying data of Fundamentals.exchange_id
# is of type string, .latest returns a Classifier
# market为整数类型，代表市场板块（上海主板、创业板等
exchange = Fundamentals.info.market.latest

In [3]:
type(exchange)

zipline.pipeline.classifiers.classifier.Latest

以前，我们看到`latest`的属性产生了一个`Factor`的实例。在以上情况下，由于底层数据是字符串类型，因此`latest`产生分类器。

类似地，计算证券`latest`晨星部门代码生成分类器。在这种情况下，基础类型是一个`int`，但整数不代表数值（它是一个类别），因此它会生成一个分类器。要获得`latest`晨星部门代码，我们可以使用内置的`Sector`分类器。

In [4]:
#from quantopian.pipeline.classifiers.fundamentals import Sector
morningstar_sector = Fundamentals.info.sector_code.latest

使用`Sector`等同于`Fundamentals.morningstar_sector_code.latest`。

## Building Filters from Classifers

分类器也使用`isnull`，`eq`和`startswith`等方法生成过滤器。生成过滤器的分类器方法的完整列表可以在[这里](https://www.quantopian.com/help#quantopian_pipeline_classifiers_Classifier)找到。

作为一个例子，如果我们想要一个过滤器来选择在纽约证券交易所交易的证券，我们可以使用交易分类器的eq方法。

In [5]:
# 查询代码含义
Fundamentals.market_cname(3)

'深圳主板'

In [6]:
# 上海主板过滤
sse_filter = exchange.eq(0)

对于有'NYS'作为其最近exchange_id的证券，此过滤器将返回True。

## Quantiles

分类器也可以由各种因子方法生产。其中最通用的是分位数方法，它接受一个二进制数作为参数。分位数分类器为因子输出中的每个非`NaN`数据点分配一个从0到（bin-1）的标签。`NaN`标记为-1。别名可用于四分位数（quantiles(4)），五分位数（quantiles(5)）和十位数（quantiles(10)）。 作为一个例子，这是一个因子的`top`十分之一过滤器可能是这样的：

让我们将每个分类器放入一个管道中，然后运行它以查看它们的外观。

In [7]:
def make_pipeline():

    exchange = Fundamentals.info.market.latest
    # 深圳主板过滤器
    szse_filter = exchange.eq(3)

    morningstar_sector = Fundamentals.info.sector_code.latest

    dollar_volume_decile = AverageDollarVolume(
        window_length=10,
        inputs=[USEquityPricing.close, USEquityPricing.volume]).deciles()
    top_decile = (dollar_volume_decile.eq(9))

    return Pipeline(
        columns={
            'exchange': exchange,
            'sector_code': morningstar_sector,
            'dollar_volume_decile': dollar_volume_decile
        },
        screen=(szse_filter & top_decile))

In [8]:
result = run_pipeline(make_pipeline(), '2015-05-05', '2015-05-05')
print('Number of securities that passed the filter: %d' % len(result))
result.head()

Number of securities that passed the filter: 63


dollar_volume_decile  exchange  \
2015-05-05 00:00:00+00:00 平安银行(000001)                     9         3   
                          万 科Ａ(000002)                     9         3   
                          中国宝安(000009)                     9         3   
                          中粮地产(000031)                     9         3   
                          中集集团(000039)                     9         3   

                                        sector_code  
2015-05-05 00:00:00+00:00 平安银行(000001)        103.0  
                          万 科Ａ(000002)        104.0  
                          中国宝安(000009)        310.0  
                          中粮地产(000031)        104.0  
                          中集集团(000039)        310.0

分类器也可用于描述因子输出上复杂变换的分组键。 分组操作（如`demean`和`rank`）超出了本教程的范围。 未来的教程将涵盖分类器的更高级用途。

在下一课中，我们将看看我们可以在流水线中使用的不同数据集。